Cells below were created as notes while watching Andrej Karpathy's video "[Let's build GPT: from scratch, in code, spelled out.
](https://www.youtube.com/watch?v=kCc8FmEb1nY)". Check it out if you haven't!


### Tokens - encoding text to a numeric format


In [74]:
from transformers import GPT2Tokenizer

# This is the tokenizer used by GPT-2.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

print(f"Vocab size: {tokenizer.vocab_size}")


Vocab size: 50257


In [75]:
test_str = "A rather long text to demonstrate the tokenizer. Coool, right?"

# GPT-2 used a subword tokenizer, meaning that each token corresponds to part of a word
str_enc = tokenizer.encode(test_str)  # Tokenized string
print([tokenizer.decode([s]) for s in str_enc])
print(str_enc)


['A', ' rather', ' long', ' text', ' to', ' demonstrate', ' the', ' token', 'izer', '.', ' Co', 'ool', ',', ' right', '?']
[32, 2138, 890, 2420, 284, 10176, 262, 11241, 7509, 13, 1766, 970, 11, 826, 30]


In [76]:
# Note that tokens correspond to subwords. Because of this, the encoded sequence has more tokens compared to the number of words in the encoded text.
print(f"Length of text: {len(test_str.split(' '))}")
print(f"Length of encoded seq: {len(str_enc)}")


Length of text: 10
Length of encoded seq: 15


For this example, we will be using text from Shakespeare as our corpus.


In [77]:
# Load data
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text[:300])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


We will be using a very simple tokenization scheme - encoding single characters as tokens.

Therefore, our vocabulary will consist of all symbols used in the text.


In [78]:
vocab = sorted(list(set(text)))
print("".join(vocab))

vocab_size = len(vocab)
print(f"Vocab size: {vocab_size}")



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


Note that our vocabulary is much smaller compared to the GPT-2 tokenizer. Keep this in mind as we continue!


In [79]:
# Simple tokenization scheme by using the character's index in the vocabulary as its token.
stoi = {ch: i for i, ch in enumerate(vocab)}  # string-to-integer
itos = {i: ch for i, ch in enumerate(vocab)}  # integer-to-string

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])


Let's encode our original example text again, now using this simple tokenizer


In [80]:
print(encode(test_str))
print(decode(encode(test_str)))


[13, 1, 56, 39, 58, 46, 43, 56, 1, 50, 53, 52, 45, 1, 58, 43, 62, 58, 1, 58, 53, 1, 42, 43, 51, 53, 52, 57, 58, 56, 39, 58, 43, 1, 58, 46, 43, 1, 58, 53, 49, 43, 52, 47, 64, 43, 56, 8, 1, 15, 53, 53, 53, 50, 6, 1, 56, 47, 45, 46, 58, 12]
A rather long text to demonstrate the tokenizer. Coool, right?


Let's check how our encoded sequence compares to the original text in length now.


In [81]:
print(f"Length of text: {len(test_str.split(' '))}")
print(f"Length of encoded seq: {len(encode(test_str))}")


Length of text: 10
Length of encoded seq: 62


See how it is much longer? Since we are using a smaller vocabulary, we must use more tokens to encode our sequences.

This shows the inherent relationship between vocabulary size and sequence length - a smaller vocabulary results in longer sequences.


In [82]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [83]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


In [84]:
block_size = 8
train_data[: block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

### Bigram model

Let's build arguably the simplest language model possible - a bigram model, which predicts the next token based on the previous token only. This means that we are modeling our text as Markov process, where the probability of the next state (token) only depends on the present state (the previous token),


In [85]:
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


In [86]:
import torch.nn as nn
import torch.nn.functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()

        self.embedding = torch.nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.embedding(idx)

        if targets is not None:
            B, T, C = logits.shape
            logits_ = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits_, targets)
        else:
            loss = None

        return logits, loss

    def generate(self, idx, num_steps):
        for _ in range(num_steps):
            logits, _ = self.forward(idx)
            logits = logits[:, -1, :]

            # Note: Sampling is probabilistic.
            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


model = BigramLanguageModel(vocab_size)


In [87]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

for iter in range(10000):
    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)

    loss.backward()
    optimizer.step()

    if iter % 1000 == 0:
        print(f"Train loss: {loss}")


Train loss: 4.784310817718506
Train loss: 3.5806233882904053
Train loss: 2.2971673011779785
Train loss: 2.68766450881958
Train loss: 2.5791971683502197
Train loss: 2.551861524581909
Train loss: 2.7095766067504883
Train loss: 3.2053205966949463
Train loss: 2.8106496334075928
Train loss: 2.5617117881774902


In [88]:
output = model.generate(torch.zeros((1, 1), dtype=torch.long), 1000)

print(decode(output[0].tolist()))



y me the,
And
BEE:

Corear car hyothed:
NETI:
TI
A alury cerenend,
INAf; winde g wald slul bun whe

K:
Houker
Byo he kee ty,

IEThout st:


G sayomatondgrende'TI'lanthinin: ans nd s

He re.
LUShoree, somarn I's temy kethiersptersureyond myou ftr for g o walos nt kit on d.

Bulondstyonse indis prentous oodod.
toupreaco llet if tht

Thteais;
Malesheais whe; linon p nde yomim d n,
TIVI poowour w h nig har-----rin lis s ar:
Gereitho wond bu ack'
G nghat held inghe beromeas,
ARE y as.
Wht fe, heale fon:
Berernkit stoncet mollar dir ce s:

Fam OLALElodo the horerthe IOPrure se,
LI bor d,

CUSwamadr t k is chaverofithanontowng boute wantorr ithe,
The l what worathalichor f shecthipr dowserll! cat be he oong st,
Upl, ifrenit w fo swooul.
US ond, it le litene,
II bur e Win'soursce kelind seail thit:
War hee pend; YOLUE:


As, aurntated ay l barfacee hes ndetasuse

IIONGLLInthe Wie tan dan mbe, mang sue u wengesitor wis.
Fo dat
IUS:
ANUK:
Fathtethemyor owelen.
I mikece;
LALom hat beexcer ingr w

Not that bad, but far from perfect.


Naive way to predict next token is by using the mean of all previous token's embeddings.

$$ \hat{x*t} = f(\Sigma*{i=0}^{t-1} x_i) $$


In [89]:
import torch

torch.manual_seed(1337)

T, C = 6, 4

# Let's create a mock sequence of embedded tokens. Let's give each row incrementing values to more easily see what happens.
x = torch.arange(1, T + 1).view(-1, 1).repeat(1, C).float()

print(x.shape)
print(x)


torch.Size([6, 4])
tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [4., 4., 4., 4.],
        [5., 5., 5., 5.],
        [6., 6., 6., 6.]])


In [90]:
# Slow way, for-loop...


def mean(x):
    xbow = torch.zeros((T, C))

    for t in range(T):  # For each time step in seq.
        xprev = x[: t + 1, :]
        xbow[t] = torch.mean(xprev, dim=0)

    return xbow


print(mean(x))


tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.5000, 1.5000, 1.5000, 1.5000],
        [2.0000, 2.0000, 2.0000, 2.0000],
        [2.5000, 2.5000, 2.5000, 2.5000],
        [3.0000, 3.0000, 3.0000, 3.0000],
        [3.5000, 3.5000, 3.5000, 3.5000]])


In [91]:
# Fast way - matrix mult!

wei = torch.tril(torch.ones(T, T))  # Create lower triangular matrix.
print(wei)


tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [92]:
# When matmuled with sequence with shape (T, C), we almost get what we want - we get the sum of the rows, but not the mean.
print(wei @ x)


tensor([[ 1.,  1.,  1.,  1.],
        [ 3.,  3.,  3.,  3.],
        [ 6.,  6.,  6.,  6.],
        [10., 10., 10., 10.],
        [15., 15., 15., 15.],
        [21., 21., 21., 21.]])


In [93]:
# To get the mean instead, we normalize each row to sum to 1.
row_norm = wei.sum(dim=1, keepdim=True)
wei = wei / row_norm

print(wei @ x)


tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.5000, 1.5000, 1.5000, 1.5000],
        [2.0000, 2.0000, 2.0000, 2.0000],
        [2.5000, 2.5000, 2.5000, 2.5000],
        [3.0000, 3.0000, 3.0000, 3.0000],
        [3.5000, 3.5000, 3.5000, 3.5000]])


Let's compare speeds


In [94]:
%%timeit
wei @ x 

2.15 µs ± 53 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [95]:
%%timeit
mean(x)


79.6 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


More than 10x faster, and we are not even using the GPU - not bad


Now, there is one more way to achieve the same outcome using softmax across each row, which normalizes it to sum to 1. To mask out future tokens, we replace their values by -inf first.


In [96]:
tri = torch.tril(torch.ones(T, T))

wei = torch.ones((T, T))
wei = wei.masked_fill(tri == 0, -torch.inf)
wei = F.softmax(wei, dim=1)

print(wei)
print(wei @ x)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]])
tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.5000, 1.5000, 1.5000, 1.5000],
        [2.0000, 2.0000, 2.0000, 2.0000],
        [2.5000, 2.5000, 2.5000, 2.5000],
        [3.0000, 3.0000, 3.0000, 3.0000],
        [3.5000, 3.5000, 3.5000, 3.5000]])


The reason this is interesting is because this approach works regardless of the initial values of the weights matrix.

We can fill it with random values and still get rows that are normalized to 1:


In [97]:
wei = torch.randn((T, T))
wei = wei.masked_fill(tri == 0, -torch.inf)
wei = F.softmax(wei, dim=1)

wei


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7089, 0.2911, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5166, 0.3210, 0.1624, 0.0000, 0.0000, 0.0000],
        [0.1616, 0.3187, 0.1423, 0.3774, 0.0000, 0.0000],
        [0.3024, 0.1149, 0.0874, 0.4666, 0.0287, 0.0000],
        [0.1292, 0.1173, 0.1132, 0.0494, 0.1582, 0.4326]])

TODO: Describe how this can be intepreted as affinities, or coupling, between tokens.


## Self-attention


Now, can we do better than a simple average of the previous tokens? Indeed we can! We can do a weighted sum, where the weights represent the importance of each token. Even better, we can do a _learned_ weighted sum, where the weights are data-dependent.

This is in fact exactly what the mechanism underlying transformers does, so called _self-attention_. It works like this:

1. Each token emits three values: 1) a query _Q_, telling other tokens what it is "looking for", 2) a key _K_, telling other tokens what its own identity, and 3) a value _V_, which is the value it emits if interacted with. These are all outputs of learned functions, typically `nn.Linear` layers.
2. Every token's query is multiplied through a dot-product with every other token's key, giving the _attention weights_ alpha
3. The attention weights are used to do a weighted sum of the values to produce the output features

The image below shows a single token's query interacting with the keys and values of the other tokens:

![Alt text](./images/attention_example.svg)


WIP Describe scaled dot-product attention

![Alt text](./images/scaled_dot_product_attn.svg)


In [103]:
head_size = 32

x = torch.randn(T, C)

key = nn.Linear(C, head_size)
query = nn.Linear(C, head_size)
value = nn.Linear(C, head_size)

k = key(x)  # (T, head_size)
q = query(x)  # (T, head_size)
v = value(x)  # (T, head_size)

wei = q @ k.T  # (T, head_size) @ (head_size, T) = (T, T)
wei = wei.masked_fill(tri == 0, -torch.inf)
wei = F.softmax(wei, dim=1)

print(wei)
print(wei @ v)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0255, 0.9745, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1819, 0.8151, 0.0030, 0.0000, 0.0000, 0.0000],
        [0.0498, 0.0073, 0.9410, 0.0018, 0.0000, 0.0000],
        [0.0122, 0.2620, 0.0863, 0.0065, 0.6330, 0.0000],
        [0.1704, 0.1016, 0.0054, 0.6455, 0.0112, 0.0659]],
       grad_fn=<SoftmaxBackward0>)
tensor([[-0.2413,  0.0150,  0.1314,  0.8664, -0.0139, -0.2534, -0.3954, -0.1627,
         -0.1547, -0.1018, -0.0184,  0.5015, -0.3924,  0.1592,  0.6902, -0.7130,
          0.1581,  0.5846, -0.6353, -0.4605,  0.2287, -0.1649, -0.1271, -0.3449,
         -0.1098, -0.3847,  0.7201, -0.0903, -0.2255, -0.1913, -0.2528,  0.2243],
        [-1.1017, -0.3192, -0.5417,  1.0453,  0.6213, -0.1645,  0.2073, -0.4482,
         -1.4633, -0.5932,  0.7303, -0.0174, -0.8551,  0.5447, -0.9249, -0.9371,
         -0.6455,  0.8439, -0.1429,  0.3908, -0.1980, -0.4642, -1.0998, -0.7725,
          0.0647,  0.4937, -0.5857, -0.6920, -0.

## Transformer implementation
